In [1]:
import random
from collections import defaultdict

import pandas as pd
import swifter  # noqa: F401
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from tqdm.notebook import tqdm

In [2]:
MIN_COMMENTS_PER_SUBSET = 50
N_OF_SPLITS_PER_SUBSET = 50

In [3]:
df_reddit = pd.read_parquet("../data/processed/reddit_posts.parquet")
df_reddit["created"] = pd.to_datetime(df_reddit["created"], format="%Y-%m-%d %H:%M:%S")
df_reddit["edited"] = pd.to_datetime(df_reddit["edited"], format="%Y-%m-%d %H:%M:%S")

df_reddit.head(1)

,id,parent_id,author,body,created,depth,edited,score,search_query,subreddit,title,url,num_comments
0,yxu5tv,<NA>,magus-21,"Secretly lending customer funds, market-making...",2022-11-17 16:10:14,-1,NaT,1597,Safe Moon,r/CryptoCurrency,"""DYOR"" is worthless. You can't ""Do Your Own Re...",https://www.reddit.com/r/CryptoCurrency/commen...,634


In [4]:
df_coins = pd.read_json("../data/raw/coins.json")
df_coins["start_date"] = pd.to_datetime(df_coins["start_date"], format="ISO8601")
df_coins["end_date"] = pd.to_datetime(df_coins["end_date"], format="ISO8601")
df_coins = df_coins.set_index("name")

df_coins.head(1)

,symbol,fraud,test,max_market_cap_e9,start_date,subreddits,end_date
name,,,,,,,
Bitcoin,BTC,False,False,1800.0,2010-07-14,"[CryptoCurrency, CryptoMoonShots, CryptoMarket...",NaT


In [5]:
def get_number_of_comments_per_coin(name: str, cutoff_date: str) -> int:
    """
    Get the number of comments for a given coin.

    Args:
        name (str): The name of the coin.
        cutoff_date (str): The cutoff date for the comments.

    Returns:
        int: The number of comments for the given coin.

    """
    cutoff_date = pd.to_datetime(cutoff_date, format="%Y-%m-%d")
    return len(df_reddit[(df_reddit["search_query"] == name) & (df_reddit["created"] <= cutoff_date)])


def get_first_comment_date(name: str) -> str:
    """
    Get the first comment date for a given coin.

    Args:
        name (str): The name of the coin.

    Returns:
        str: The first comment date for the given coin.

    """
    return df_reddit[df_reddit["search_query"] == name]["created"].min()


def get_last_comment_date(name: str) -> str:
    """
    Get the last comment date for a given coin.

    Args:
        name (str): The name of the coin.

    Returns:
        str: The last comment date for the given coin.

    """
    mod_df_reddit = df_reddit.copy()
    mod_df_reddit["edited"] = mod_df_reddit["edited"].fillna(mod_df_reddit["created"])
    return mod_df_reddit[mod_df_reddit["search_query"] == name]["edited"].max()


def get_coin_info(name: str) -> pd.Series:
    """
    Get coin information for a given coin name.

    Args:
        name (str): The name of the coin.

    Returns:
        pd.Series: The coin information.

    """
    return df_coins.loc[name]


def get_coin_info_row(row: pd.Series) -> pd.Series:
    """
    Get coin information for a given row.

    Args:
        row (pd.Series): A row from the dataframe containing a search_query column.

    Returns:
        pd.Series: The coin information corresponding to the search_query.

    """
    return df_coins.loc[row.search_query]


def is_comment_valid_row(row: pd.Series) -> bool:
    """
    Check if a comment is valid based on the coin's start and end dates.

    Args:
        row (pd.Series): A row from the dataframe containing created and edited columns.

    Returns:
        bool: True if the comment is valid, False otherwise.

    """
    coin = get_coin_info_row(row)

    comment_date = row.edited if pd.notna(row.edited) else row.created
    start_date_valid = comment_date >= coin["start_date"]
    end_date_valid = comment_date <= coin["end_date"] if pd.notna(coin["end_date"]) else True

    return start_date_valid and end_date_valid

In [6]:
df_reddit["test"] = df_reddit.swifter.apply(get_coin_info_row, axis=1)["test"]
df_reddit["valid"] = df_reddit.swifter.apply(is_comment_valid_row, axis=1)

df_reddit = df_reddit[df_reddit["valid"]]
df_reddit = df_reddit.drop(columns=["valid"])
df_reddit = df_reddit.reset_index(drop=True)

df_reddit.head(1)

Dask Apply:   0%|          | 0/49 [00:00<?, ?it/s]

Dask Apply:   0%|          | 0/49 [00:00<?, ?it/s]

,id,parent_id,author,body,created,depth,edited,score,search_query,subreddit,title,url,num_comments,test
0,yxu5tv,<NA>,magus-21,"Secretly lending customer funds, market-making...",2022-11-17 16:10:14,-1,NaT,1597,Safe Moon,r/CryptoCurrency,"""DYOR"" is worthless. You can't ""Do Your Own Re...",https://www.reddit.com/r/CryptoCurrency/commen...,634,True


In [7]:
df_reddit["search_query"].value_counts()

search_query
Bitcoin       142163
Chainlink      72875
Ethereum       72409
Safe Moon      69352
Cosmos         57082
Avalanche      35673
FTX Token      21656
THORChain      18772
Terra Luna      8683
BitForex        2611
BeerCoin         805
Name: count, dtype: Int64

In [8]:
train_df = df_reddit[~df_reddit["test"]]
test_df = df_reddit[df_reddit["test"]]

In [9]:
unique_coins = df_reddit["search_query"].unique().tolist()

In [10]:
# Seed for reproducibility
random.seed(42)

# Create subsets
list_subsets = []
for coin in unique_coins:
    # Get metadata for the coin
    metadata_coin = get_coin_info(coin).copy()
    metadata_coin["start_date"] = get_first_comment_date(coin)
    metadata_coin["end_date"] = get_last_comment_date(coin)

    # Create N_OF_SPLITS_PER_SUBSET splits
    for _ in range(N_OF_SPLITS_PER_SUBSET):
        successful_split = False
        while not successful_split:
            # Get a random date between the start and end date
            random_day_diff = random.randint(0, (metadata_coin["end_date"] - metadata_coin["start_date"]).days)
            random_date = metadata_coin["start_date"].date() + pd.Timedelta(days=random_day_diff)

            # Check if there are enough comments for the subset to be valid
            if get_number_of_comments_per_coin(coin, random_date) > MIN_COMMENTS_PER_SUBSET:
                successful_split = True

        # Append the subset to the list
        list_subsets.append({"coin": coin, "split_date": random_date, "fraud": metadata_coin["fraud"]})

In [11]:
# Drop duplicates
df_subsets = pd.DataFrame(list_subsets)
df_subsets = df_subsets.drop_duplicates()
df_subsets

,coin,split_date,fraud
0,Safe Moon,2022-05-13,True
1,Safe Moon,2022-02-13,True
2,Safe Moon,2022-10-27,True
3,Safe Moon,2022-09-26,True
4,Safe Moon,2022-09-04,True
...,...,...,...
545,BitForex,2022-07-07,True
546,BitForex,2023-10-22,True
547,BitForex,2021-07-20,True
548,BitForex,2023-09-11,True


In [12]:
df_subsets.coin.value_counts()

coin
Chainlink     50
THORChain     50
Ethereum      50
BitForex      50
Bitcoin       50
Terra Luna    48
Cosmos        48
Avalanche     47
FTX Token     47
Safe Moon     46
BeerCoin      21
Name: count, dtype: int64

In [13]:
train_coins = train_df.search_query.unique().tolist()
df_subsets_train = df_subsets[df_subsets["coin"].isin(train_coins)]
df_subsets_test = df_subsets[~df_subsets["coin"].isin(train_coins)]

In [14]:
df_subsets_train.coin.value_counts()

coin
Chainlink     50
THORChain     50
Bitcoin       50
BitForex      50
Terra Luna    48
Avalanche     47
BeerCoin      21
Name: count, dtype: int64

In [15]:
vectorizer = CountVectorizer(
    stop_words="english",
    analyzer="word",
    lowercase=True,
)

vectorizer.fit(train_df["body"])

CountVectorizer(stop_words='english')

In [16]:
print("Number of features:", len(vectorizer.vocabulary_))

Number of features: 95630


In [17]:
def get_text_from_split(data: pd.DataFrame, split_date: str, coin: str) -> str:
    """
    Get the text data for a given coin and split date.

    Args:
        data (pd.DataFrame): The dataframe containing the data.
        split_date (str): The split date.
        coin (str): The coin name.

    Returns:
        str: The text data for the given coin and split date

    """
    split_date = pd.to_datetime(split_date, format="%Y-%m-%d")

    data = data.copy()
    data["edited"] = data["edited"].fillna(data["created"])
    data = data[data["search_query"] == coin]
    data = data[data["edited"] <= split_date]
    return data["body"].str.cat(sep=" ")

In [18]:
# LOOCV based on each coin
metrics = defaultdict(dict)

for i, coin in tqdm(enumerate(df_subsets_train.coin.unique()), total=len(df_subsets_train.coin.unique())):
    # Get the fitting and validation splits
    fit_df = df_subsets_train[df_subsets_train["coin"] != coin]
    val_df = df_subsets_train[df_subsets_train["coin"] == coin]

    # Get the fitting data
    list_fitting_data = []
    for _, row in fit_df.iterrows():
        list_fitting_data.append(
            {
                "coin": row["coin"],
                "split_date": row["split_date"],
                "text": get_text_from_split(train_df, row["split_date"], row["coin"]),
                "fraud": row["fraud"],
            },
        )

    # Get the validation data
    list_validation_data = []
    for _, row in val_df.iterrows():
        list_validation_data.append(
            {
                "coin": row["coin"],
                "split_date": row["split_date"],
                "text": get_text_from_split(train_df, row["split_date"], row["coin"]),
                "fraud": row["fraud"],
            },
        )

    # Create the fitting and validation dataframes
    df_fitting = pd.DataFrame(list_fitting_data)
    df_validation = pd.DataFrame(list_validation_data)
    del list_fitting_data
    del list_validation_data

    # fitting Multinomial Naive Bayes
    clf = MultinomialNB()
    X_fitting = vectorizer.transform(df_fitting["text"])
    y_fitting = df_fitting["fraud"]
    clf.fit(X_fitting, y_fitting)

    # Predicting the validation set
    X_validation = vectorizer.transform(df_validation["text"])
    y_validation = df_validation["fraud"].to_numpy()
    y_pred = clf.predict_proba(X_validation)

    # calculating metrics for left out coin
    metrics[i] = {
        "coin_left_out": coin,
        "coin_split_date": df_validation["split_date"].to_list(),
        "coin_fraud": df_validation["fraud"].to_list(),
        "accuracy": 1 * (y_pred[:, 1].round() == y_validation),
        "y_pred": y_pred[:, 1].round(3),
        "y_true": y_validation,
    }

    # deleting dataframes to free up memory
    del df_fitting
    del df_validation

  0%|          | 0/7 [00:00<?, ?it/s]

In [19]:
metrics_df = pd.DataFrame.from_dict(metrics, orient="index")
metrics_df = metrics_df.set_index("coin_left_out")
metrics_df = metrics_df.explode(["coin_split_date", "coin_fraud", "accuracy", "y_pred", "y_true"])
metrics_df.groupby("coin_left_out").accuracy.mean().sort_values(ascending=False)

coin_left_out
Avalanche          1.0
Bitcoin            1.0
Chainlink          1.0
THORChain          1.0
BeerCoin      0.809524
BitForex           0.0
Terra Luna         0.0
Name: accuracy, dtype: object

In [20]:
# Get the training data
list_train_data = []
for _, row in df_subsets_train.iterrows():
    list_train_data.append(
        {
            "coin": row["coin"],
            "split_date": row["split_date"],
            "text": get_text_from_split(train_df, row["split_date"], row["coin"]),
            "fraud": row["fraud"],
        },
    )

df_training = pd.DataFrame(list_train_data)
del list_train_data

# fitting Multinomial Naive Bayes
clf = MultinomialNB()
X_fitting = vectorizer.transform(df_training["text"])
y_fitting = df_training["fraud"]
clf.fit(X_fitting, y_fitting)

# deleting dataframes to free up memory
del df_training


for coin in tqdm(df_subsets_test.coin.unique()):
    # Get the test data
    list_test_data = []
    for _, row in df_subsets_test[df_subsets_test["coin"] == coin].iterrows():
        list_test_data.append(
            {
                "coin": row["coin"],
                "split_date": row["split_date"],
                "text": get_text_from_split(test_df, row["split_date"], row["coin"]),
                "fraud": row["fraud"],
            },
        )

    df_test = pd.DataFrame(list_test_data)
    del list_test_data

    # Predicting the validation set
    X_validation = vectorizer.transform(df_test["text"])
    y_validation = df_test["fraud"].to_numpy()
    y_pred = clf.predict_proba(X_validation)

    print(f"""
    Coin left out: {coin}
    Correct Label: {df_test['fraud'].iloc[0]}
    Predicted as Fraud: {int(y_pred[:, 1].sum())}
    Predicted as Not Fraud: {int(len(y_pred) - y_pred[:, 1].sum())}
    Mean prediction: {y_pred[:, 1].mean():.4f}
    Accuracy: {1 * (y_pred[:, 1].round() == y_validation).mean():.4f}
    """)

  0%|          | 0/4 [00:00<?, ?it/s]


    Coin left out: Safe Moon
    Correct Label: True
    Predicted as Fraud: 0
    Predicted as Not Fraud: 46
    Mean prediction: 0.0000
    Accuracy: 0.0000
    

    Coin left out: FTX Token
    Correct Label: True
    Predicted as Fraud: 0
    Predicted as Not Fraud: 47
    Mean prediction: 0.0000
    Accuracy: 0.0000
    

    Coin left out: Cosmos
    Correct Label: False
    Predicted as Fraud: 0
    Predicted as Not Fraud: 48
    Mean prediction: 0.0000
    Accuracy: 1.0000
    

    Coin left out: Ethereum
    Correct Label: False
    Predicted as Fraud: 0
    Predicted as Not Fraud: 50
    Mean prediction: 0.0000
    Accuracy: 1.0000
    
